## Recursively delete group

In [1]:
import pymysql

# Create a MySQL database connection
conn = pymysql.connect(
    host='ls-4b4f6e549c1e57f75963fa2606807ebca3320b7f.c4132jfwzbkh.ap-south-1.rds.amazonaws.com',
    user='dbmasteruser',
    password='87v9hRn^6',
    database='netcredential',
    port=3306
)

# Create a cursor
cursor = conn.cursor()

def delete_group_and_related(group_ids):
    for group_id in group_ids:
        # Delete all events associated with the group
        cursor.execute(f"DELETE FROM events WHERE group_id = '{group_id}'")

        # Delete all credentials associated with the group
        cursor.execute(f"DELETE FROM credentials WHERE group_id = '{group_id}'")

        # Finally, delete the group itself
        cursor.execute(f"DELETE FROM `groups` WHERE id = '{group_id}'")

    # Commit the changes
    conn.commit()

# Usage: Provide the group_id you want to delete
group_ids_to_delete = ["3sdABFlBgJJzFvvtxRLl", "D1r94mnzxqRtdLukCmO7", "d6N1CmtroEhqVlXUEldM", "dveyiT23Tu7ruHAYsA0G", "HjJCt92qBO6VrUj5GeWA", "jf35zOum1Lj5DOmpOa84", "JGVImzaSepAYy4lzT3W5", "Lt3bUvVJO58PwbuMBy0R", "Lt6SxiRdHw6QODmTfK59", "r0KEnzj76Eq0rX3yUmFr", "T5iWzaUNPWWA0tOKzZNn", "Tn0YOcKg77oxUkpo8mj8", "tVaoOhDWxz9MNymYmegO", "yrTPuvCDKLK4w8m6mT8S"]
delete_group_and_related(group_ids_to_delete)

# Close the cursor and database connection
cursor.close()
conn.close()

In [5]:
import pymysql
import random
import string

# Function to generate a unique string ID
def generate_design_id(length=12):
    alphabet = string.ascii_letters + string.digits
    return ''.join(random.choice(alphabet) for _ in range(length))

# Database connection parameters
connection = pymysql.connect(
    host='ls-4b4f6e549c1e57f75963fa2606807ebca3320b7f.c4132jfwzbkh.ap-south-1.rds.amazonaws.com',
    user='dbmasteruser',
    password='87v9hRn^6',
    database='netcredential',
    port=3306
)

try:
    with connection.cursor() as cursor:
        # SQL query to select data from Group table
        select_query = "SELECT id, org_id, name, template, timestamp FROM `groups`"

        # Prepare the insert statement for Design table
        insert_query = """
            INSERT INTO designs (id, org_id, name, status, canvas, items, attributes, preview, template, timestamp, updated)
            VALUES (%s, %s, %s, 'Legacy', NULL, NULL, NULL, NULL, %s, %s, %s)
        """

        # Prepare the update statement for Group table
        update_query = "UPDATE `groups` SET template = %s WHERE id = %s"

        # Execute the select query
        cursor.execute(select_query)
        groups = cursor.fetchall()

        # Loop through each row in Group table
        for group in groups:
            # Generate a unique design ID
            design_id = generate_design_id()
            
            print(group['org_id'])

            # Prepare data for insertion into Design table
            data = (design_id, group['org_id'], group['name'], group['template'], group['timestamp'], group['timestamp'])

            # Execute the insert query for Design table
            cursor.execute(insert_query, data)

            # Prepare data for updating Group table
            update_data = (design_id, group['id'])

            # Execute the update query for Group table
            cursor.execute(update_query, update_data)

        # Commit the changes
        connection.commit()

except pymysql.MySQLError as error:
    print("Error: ", error)

finally:
    # Close the database connection
    connection.close()

TypeError: tuple indices must be integers or slices, not str

## Resend Queued Credentials (Group)

In [1]:
import pymysql
import requests
import json

# Create a MySQL database connection
connection = pymysql.connect(
    host='ls-4b4f6e549c1e57f75963fa2606807ebca3320b7f.c4132jfwzbkh.ap-south-1.rds.amazonaws.com',
    user='dbmasteruser',
    password='87v9hRn^6',
    database='netcredential',
    port=3306,
    cursorclass=pymysql.cursors.DictCursor
)

org = "Meta Craftlab"
orgemail = "hello@craftlab.ai"
groupname = "Mastering Svelte Fundamentals"

try:
    with connection.cursor() as cursor:
        select_query = "SELECT * FROM `credentials` WHERE group_id = 'AgvrpokQ3mikFXZhtWfd90w1Ycvv'"

        cursor.execute(select_query)
        credentials = cursor.fetchall()

        for credential in credentials:
            response = requests.post("https://api.mailgun.net/v3/mail.netcredential.com/messages",
                auth = ("api", "7a8f33f22d9d47885f95b574837a2ca9-9dfbeecd-2f6df695"),
                data = {"from": org + " via NetCredential <noreply@mail.netcredential.com>",
                            "to": [f"{credential['recipient_name']} <{credential['recipient_email']}>"],
                            "subject": groupname + " from " + org,
                            "o:tag": [org],
                            "v:credid": credential['id'],
                            "h:Reply-To": f"{org} <{orgemail}>",
                            "recipient-variables": json.dumps({credential['recipient_email']: {"name":credential['recipient_name'].split(" ")[0], "company":org, "credid":credential['id']}}),
                            "template": "credential"})
            print(credential['recipient_email'], response.json())


except pymysql.MySQLError as error:
    print("Error: ", error)

finally:
    # Close the database connection
    connection.close()

saurabhcmalviya@gmail.com {'id': '<20240616092124.37499c54bd7ae5fc@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
Kaneezfatimabohra@gmail.com {'id': '<20240616092125.32fea062fa3e1666@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
chinmay.pimpalgaonkar.cyb@ghrce.raisoni.net {'id': '<20240616092127.d62f324537772c8c@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
thondanandini@gmail.com {'id': '<20240616092128.ad150977e13bdd38@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
miyajiiqra@gmail.com {'id': '<20240616092129.f8667123a1165a9f@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
csarthak013@gmail.com {'id': '<20240616092130.27cff871c5578d0a@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
ys.prasadcode@gmail.com {'id': '<20240616092131.fedbece519f16413@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
priyantbanerjee@gmail.com {'id': '<20240616092132.bb0337892ffc274a@mail.netcredential.com>', 'message': 'Queued

dalima.sahu@ssipmt.com {'id': '<20240616092241.b01211f289074d4c@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
sheshank.singh@vit.edu.in {'id': '<20240616092242.375e0dc5592be070@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
monikakonde41@gmail.com {'id': '<20240616092243.3aebafe3f7162a6e@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
vaibhavrathod2282@gmail.com {'id': '<20240616092244.8097cdad0c04975e@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
olivedaniel24@gmail.com {'id': '<20240616092246.17bc1952af30959e@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
kishankumarsahu120@gmail.com {'id': '<20240616092247.782fd68b44a29204@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
shaikhsarfaraj3940@gmail.com {'id': '<20240616092248.4f78f245229f1098@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
nikhilsarve05032002@gmail.com {'id': '<20240616092249.a99081d9cf3287d6@mail.netcredential.com>', 'message': 'Queued. 

swastikasony09@gmail.com {'id': '<20240616092359.b895e5e4971273ae@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
ajayrajrthd5@gmail.com {'id': '<20240616092400.e2d2af1cd236b251@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
ankitsharma10503@gmail.com {'id': '<20240616092401.489b4f44c4ee86a3@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
21104005.swapnil.joshi@gmail.com {'id': '<20240616092402.a94c63a290715d48@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
farhan.sayed@ieee.org {'id': '<20240616092404.bc04beac3db359ab@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
yashikashukla21042003@gmail.com {'id': '<20240616092405.45557b52f1dc33e1@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
raj5patkar@gmail.com {'id': '<20240616092406.b91c93f935e6c593@mail.netcredential.com>', 'message': 'Queued. Thank you.'}
karimmirza2002@gmail.com {'id': '<20240616092407.8855de831b652b57@mail.netcredential.com>', 'message': 'Queued. Thank

## Update Legacy Design

In [19]:
import pymysql
import random
import string
import json
import requests
import base64

# Database connection parameters
connection = pymysql.connect(
    host='ls-4b4f6e549c1e57f75963fa2606807ebca3320b7f.c4132jfwzbkh.ap-south-1.rds.amazonaws.com',
    user='dbmasteruser',
    password='87v9hRn^6',
    database='netcredential',
    port=3306
)

design_id = '7yVA4ggTifaa'
canva = 'DAF6xHd_H9A/n6Z2L-STd_q_wL86y4drzA'

config = {
    'method': 'post',
    'url': 'https://us-central1-netcredential-22f17.cloudfunctions.net/generatetemplate',
    'headers': {
        'Content-Type': 'application/json'
    },
    'data': json.dumps({
        "design": f"https://www.canva.com/design/{canva}/view",
        "group": design_id
    })
}

try:
    response = requests.request(**config)
    response_data = response.json()
    template = response_data.get('template')
        
    config = {
        'method': 'post',
        'url': 'https://us-central1-netcredential-22f17.cloudfunctions.net/generatenow',
        'headers': {
            'Content-Type': 'application/json'
        },
        'data': json.dumps({
            "template": template,
            "credid": '##CREDID##',
            "view": 'image',
            "name": '##NAME##',
            "extra": {}
        })
    }
    response = requests.request(**config)
    base64_encoded_data = base64.b64encode(response.content)
    base64_image = base64_encoded_data.decode('utf-8')
        
    cursor = connection.cursor()
    
    sql = "UPDATE designs SET template = %s, preview = %s WHERE id = %s"
    cursor.execute(sql, (template, 'data:image/jpeg;base64,'+base64_image, design_id))
    
    connection.commit()

    # Close the cursor and database connection
    cursor.close()
    connection.close()
    print(template)
except Exception as error:
    print(error)

Expecting value: line 1 column 1 (char 0)


## Download PDF of all the credentials in the group

In [3]:
import pymysql
import requests
import json
import os

# Create a MySQL database connection
connection = pymysql.connect(
    host='ls-4b4f6e549c1e57f75963fa2606807ebca3320b7f.c4132jfwzbkh.ap-south-1.rds.amazonaws.com',
    user='dbmasteruser',
    password='87v9hRn^6',
    database='netcredential',
    port=3306,
    cursorclass=pymysql.cursors.DictCursor
)

group_id = "1LTqsgWb1z93D0v6vlX26HVtIJh3"

folder_path = f"./{group_id}"
os.makedirs(folder_path, exist_ok=True)

try:
    with connection.cursor() as cursor:
        select_query = f"SELECT * FROM `credentials` WHERE group_id = '{group_id}'"

        cursor.execute(select_query)
        credentials = cursor.fetchall()

        for credential in credentials:
            url = f"https://verify.netcredential.com/api/credential/{credential['id']}?view=pdf"
            response = requests.request("GET", url)
            file_path = os.path.join(folder_path, f"{credential['id']}.pdf")
            with open(file_path, 'wb') as file:
                file.write(response.content)


except pymysql.MySQLError as error:
    print("Error: ", error)

finally:
    # Close the database connection
    connection.close()